In [3]:
import os
import sys
from datetime import date, datetime

import pandas as pd
import numpy as np
import pyarrow as pa
import perspective

### Perspective Basics

TODO: brief description

In [4]:
# Set up some dummy data
data = pd.DataFrame({
    "a": np.random.rand(100),
    "b": np.ones(100),
    "c": [str(i) for i in range(100)],
    "d": [datetime.now() for i in range(100)],
    "e": [datetime.today() for i in range(100)]
})

# create a perspective.Table - the base container for data
table = perspective.Table(data)

# create a view - a query on the data
view = table.view(filter=[["a", ">", 0.5]], sort=[["a", "desc"]])

#### Getting Data

In [5]:
# get some data from the view - supported formats are dataframes, Apache Arrow, dicts of numpy arrays, row/columnar JSON, and CSV
filtered = view.to_df(end_row=10)
display(filtered)

# append to the table with new data
table.update([{"a": 1.5, "b": 1, "c": "string", "d": datetime.now()}])

# re-query the data
filtered = view.to_df(end_row=10)
display("After update", filtered)

,index,a,b,c,d,e
0,21,0.991981,1.0,21,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
1,50,0.976966,1.0,50,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
2,28,0.968338,1.0,28,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
3,30,0.967631,1.0,30,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
4,70,0.957843,1.0,70,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
5,88,0.955356,1.0,88,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
6,41,0.947526,1.0,41,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
7,57,0.938354,1.0,57,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
8,48,0.934444,1.0,48,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
9,80,0.932001,1.0,80,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821


'After update'

,index,a,b,c,d,e
0,None,1.500000,1.0,string,2020-09-03 18:42:54.076,NaT
1,21,0.991981,1.0,21,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
2,50,0.976966,1.0,50,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
3,28,0.968338,1.0,28,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
4,30,0.967631,1.0,30,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
5,70,0.957843,1.0,70,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
6,88,0.955356,1.0,88,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
7,41,0.947526,1.0,41,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
8,57,0.938354,1.0,57,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821
9,48,0.934444,1.0,48,2020-09-03 14:42:35.821,2020-09-03 14:42:35.821


#### Schemas and Updates

In [6]:
# Create an indexed table from a schema - a mapping of column names to types, primary keyed by `a`
schema_table = perspective.Table({
    "a": float,
    "b": float,
    "c": str,
    "d": datetime
}, index="a")

# Add an `on_update` callback, which fires with an Arrow-encoded binary of the updated rows
def callback(port, delta):
    # Update the indexed table with the updated rows, which will update in-place based on `index`
    schema_table.update(delta)

view.on_update(callback, mode="row")

In [7]:
# Update the unindexed table, and query the indexed table
table.update([
    {"a": 1.5, "b": 1, "c": "string", "d": datetime.now()},
    {"a": 2.5, "b": 2, "c": "string2", "d": datetime.now()}
])

# Group values by `a`
pivoted_view = schema_table.view(row_pivots=["a"], aggregates={"c": "unique"})

# Get column-oriented JSON
pivoted_output = pivoted_view.to_columns()
display(pivoted_output)

{'__ROW_PATH__': [[], [1.5], [2.5]],
 'a': [4.0, 1.5, 2.5],
 'b': [3.0, 1.0, 2.0],
 'd': [2, 1, 1],
 'c': ['-', 'string', 'string2']}

In [8]:
# Updates with the same primary key will overwrite
table.update([
    {"a": 1.5, "b": 100, "c": "new string"}
])

pivoted_output = pivoted_view.to_columns()
display(pivoted_output)

{'__ROW_PATH__': [[], [1.5], [2.5]],
 'a': [4.0, 1.5, 2.5],
 'b': [102.0, 100.0, 2.0],
 'd': [2, 1, 1],
 'c': ['-', 'new string', 'string2']}

#### TODO: Working with Arrows

### PerspectiveWidget

TODO: Describe

In [9]:
# Create a PerspectiveWidget from the tables we just created
widget = perspective.PerspectiveWidget(table, row_pivots=["a"], aggregates={"a": "avg"})
display(widget)

PerspectiveWidget(aggregates={'a': 'avg'}, columns=['index', 'a', 'b', 'c', 'd', 'e'], row_pivots=['a'])

### Chaining Perspectives with `on_update`

In [10]:
# TODO